In [1]:
import pandas as pd
import pickle
books_ratings_prepared_df = pd.read_csv("/kaggle/input/d/vadymspitsyqhbsda/book-recomendation-new/filtered_dataset_updated.csv")
books_ratings_prepared_df.shape

(152625, 3)

In [2]:
book_pivot = books_ratings_prepared_df.pivot_table(columns = 'ID', index = 'Name', values = 'Numerical_Rating')
book_pivot.fillna(0, inplace=True)
book_pivot

ID,1,2,3,4,5,6,7,8,9,12,...,10954,10959,10962,10969,10974,10977,10978,10986,10988,10996
Name,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Tis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Ways to Bug Your Parents,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10:04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11/22/63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Zero to One: Notes on Startups, or How to Build the Future",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zone One,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
with open('book_pivot.pkl', 'wb') as pivot_file:
    pickle.dump(book_pivot, pivot_file)

In [4]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
book_sparse = csr_matrix(book_pivot)
model = NearestNeighbors(algorithm = 'brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [5]:
with open('book_recommendation_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [6]:
def recommend_books(book_title, pivot_data, model, num_recommendations=25):
    try:
        row_position = pivot_data.index.get_loc(book_title)
    except KeyError:
        print(f"Book title '{book_title}' not found in the DataFrame.")
        return

    distances, suggestions = model.kneighbors(pivot_data.iloc[row_position, :].values.reshape(1, -1), n_neighbors=num_recommendations)

    print(f"\nTop {num_recommendations} Recommendations:")
    for i in range(num_recommendations):
        recommended_book_title = pivot_data.index[suggestions[0, i]]
        distance_to_recommended_book = distances[0][i]
        print(f"{i + 1}. {recommended_book_title} (Distance: {distance_to_recommended_book:.2f})")

In [7]:
book_title_to_find = "Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)"
recommend_books(book_title_to_find, book_pivot, model)


Top 25 Recommendations:
1. Harry Potter and the Prisoner of Azkaban (Harry Potter, #3) (Distance: 0.00)
2. Harry Potter and the Goblet of Fire (Harry Potter, #4) (Distance: 41.76)
3. Harry Potter and the Order of the Phoenix (Harry Potter, #5) (Distance: 48.70)
4. Harry Potter and the Half-Blood Prince (Harry Potter, #6) (Distance: 53.27)
5. Harry Potter and the Chamber of Secrets (Harry Potter, #2) (Distance: 57.55)
6. Harry Potter and the Sorcerer's Stone (Harry Potter, #1) (Distance: 59.98)
7. Harry Potter and the Deathly Hallows (Harry Potter, #7) (Distance: 87.11)
8. Eclipse (Twilight, #3) (Distance: 102.68)
9. New Moon (Twilight, #2) (Distance: 102.79)
10. Mockingjay (The Hunger Games, #3) (Distance: 103.00)
11. The Subtle Knife (His Dark Materials, #2) (Distance: 103.03)
12. Insurgent (Divergent, #2) (Distance: 103.21)
13. Breaking Dawn (Twilight, #4) (Distance: 103.27)
14. The Two Towers (The Lord of the Rings, #2) (Distance: 103.42)
15. Divergent (Divergent, #1) (Distance: 10